# Experiment: 08 NAV Compare

Compare proxy strategies across notebooks using common-period rebased NAV.

In [1]:

import os
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import shutil

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True, linewidth=140)

BASE_DIR = Path("/Users/kencharoff/workspace/projects/momentum/research")
INPUT_CSV = Path("/Users/kencharoff/workspace/projects/momentum/research/data/prices.csv")

OUTPUT_DIR = BASE_DIR / "outputs"
LOG_DIR = OUTPUT_DIR / "logs"
PLOTS_DIR = OUTPUT_DIR / "plots"
TABLES_DIR = OUTPUT_DIR / "tables"
PERF_DIR = OUTPUT_DIR / "perf"

for d in [LOG_DIR, PLOTS_DIR, TABLES_DIR, PERF_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CONFIG = {
    "input_csv": str(INPUT_CSV),
    "universe_min_coverage": 0.7,
    "rebalance": "ME",
    "cost_bps": 5.0,
    "mom_lookback_m": 12,
    "mom_skip_m": 1,
    "mom_form_m": 6,
    "beta_window_d": 252,
    "cov_lookback_m": 36,
    "ppp_eta": 2.0,
    "ppp_gamma": 10.0,
    "ppp_winner_pct": 0.3,
    "activation_span": 12,
    "trend_horizons_m": [1,3,12],
    "vol_ewma_span": 63,
    "dd_stop": 0.2,
    "stop_scale": 0.5,
    "k_top_bottom": 3,
    "comom_window_m": 36,
}

def backup_if_exists(path: Path):
    path = Path(path)
    if path.exists():
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        bak = path.with_name(path.name + f".bak.{ts}")
        shutil.copy2(path, bak)
        return bak
    return None

def safe_write(path: Path, writer_fn):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    backup_if_exists(path)
    writer_fn(path)

def normalize_field(x: str) -> str:
    s = str(x).strip().lower().replace(" ", "_")
    if s in {"adj_close", "adjclose", "adjusted_close", "adjustedclose"}:
        return "adj_close"
    if s in {"close", "open", "high", "low", "volume", "price"}:
        return s
    return s

def load_prices_csv(path: Path):
    df = pd.read_csv(path, header=None)
    row0 = df.iloc[0].astype(str).fillna("")
    row1 = df.iloc[1].astype(str).fillna("")

    date_label_rows = df.index[df.iloc[:, 0].astype(str).str.strip().str.lower() == "date"]
    date_label_idx = int(date_label_rows[0]) if len(date_label_rows) > 0 else 2
    data_start_idx = date_label_idx + 1

    def count_fields(row):
        field_set = {"adj close", "close", "open", "high", "low", "volume", "price"}
        return sum(str(v).strip().lower() in field_set for v in row)

    if row1.iloc[0].strip().lower() == "ticker":
        field_row, ticker_row = row0, row1
    elif row0.iloc[0].strip().lower() == "ticker":
        field_row, ticker_row = row1, row0
    else:
        if count_fields(row0) >= count_fields(row1):
            field_row, ticker_row = row0, row1
        else:
            field_row, ticker_row = row1, row0

    dates = pd.to_datetime(df.iloc[data_start_idx:, 0], errors="coerce")
    data = df.iloc[data_start_idx:, 1:].copy()

    cols = []
    for j in range(1, df.shape[1]):
        field = normalize_field(field_row.iloc[j])
        ticker = str(ticker_row.iloc[j]).strip()
        if ticker in ("", "nan", "None"):
            cols.append((field, None))
        else:
            cols.append((field, ticker))
    data.columns = pd.MultiIndex.from_tuples(cols)
    data = data.loc[:, [c for c in data.columns if c[0] and c[1]]]
    data = data.apply(pd.to_numeric, errors="coerce")
    data.index = dates
    data = data[~data.index.isna()]

    panel = {}
    for field in sorted(set([c[0] for c in data.columns])):
        panel[field] = data.xs(field, level=0, axis=1).copy()
    return panel

def to_month_end(px_d: pd.DataFrame) -> pd.DataFrame:
    return px_d.resample("ME").last()

def returns_from_prices(px: pd.DataFrame) -> pd.DataFrame:
    return px.pct_change()

def summarize_panel(adj_close: pd.DataFrame) -> pd.DataFrame:
    coverage = adj_close.notna().mean()
    start = adj_close.apply(lambda s: s.first_valid_index())
    end = adj_close.apply(lambda s: s.last_valid_index())
    summary = pd.DataFrame({"coverage": coverage, "start": start, "end": end})
    return summary.sort_values("coverage")

def filter_universe_by_coverage(adj_close: pd.DataFrame, min_coverage: float):
    coverage = adj_close.notna().mean()
    keep = coverage >= min_coverage
    excluded = coverage[~keep].sort_values()
    return adj_close.loc[:, keep], excluded

def map_rebalance_to_next_trading_day(rebalance_dates, trading_dates):
    rebalance_dates = pd.DatetimeIndex(rebalance_dates).sort_values()
    trading_dates = pd.DatetimeIndex(trading_dates).sort_values()
    apply_dates = []
    for t in rebalance_dates:
        idx = trading_dates.searchsorted(t, side="right")
        if idx >= len(trading_dates):
            apply_dates.append(pd.NaT)
        else:
            apply_dates.append(trading_dates[idx])
    rows = []
    for i, t in enumerate(rebalance_dates):
        start = apply_dates[i]
        if pd.isna(start):
            end = pd.NaT
        else:
            if i + 1 < len(apply_dates) and pd.notna(apply_dates[i + 1]):
                end = trading_dates[trading_dates < apply_dates[i + 1]][-1]
            else:
                end = trading_dates[-1]
        rows.append({"rebalance_date": t, "apply_start": start, "apply_end": end})
    return pd.DataFrame(rows)

def expand_weights_monthly_to_daily(w_target: pd.DataFrame, trading_dates):
    w_target = w_target.copy()
    w_target.index = pd.DatetimeIndex(w_target.index)
    mapping = map_rebalance_to_next_trading_day(w_target.index, trading_dates)
    trading_dates = pd.DatetimeIndex(trading_dates).sort_values()
    w_applied = pd.DataFrame(index=trading_dates, columns=w_target.columns, dtype=float)
    for row in mapping.itertuples(index=False):
        if pd.isna(row.apply_start):
            continue
        mask = (w_applied.index >= row.apply_start) & (w_applied.index <= row.apply_end)
        w_applied.loc[mask] = w_target.loc[row.rebalance_date].values
    w_applied = w_applied.ffill().fillna(0.0)
    return w_applied, mapping

def apply_portfolio(ret_d: pd.DataFrame, w_target: pd.DataFrame, cost_bps: float = 0.0):
    w_target = w_target.reindex(columns=ret_d.columns).fillna(0.0)
    w_applied, mapping = expand_weights_monthly_to_daily(w_target, ret_d.index)
    w_prev = w_applied.shift(1).fillna(0.0)
    r_p = (w_prev * ret_d).sum(axis=1)
    turnover = w_applied.diff().abs().sum(axis=1).fillna(0.0)
    cost = (cost_bps / 1e4) * turnover
    ret_net = r_p - cost
    nav = (1.0 + ret_net).cumprod()
    return {
        "nav": nav,
        "ret_net": ret_net,
        "turnover": turnover,
        "cost": cost,
        "w_applied_d": w_applied,
        "mapping": mapping,
    }

def metrics(nav: pd.Series, ret: pd.Series, periods_per_year: int):
    ret = ret.dropna()
    if len(ret) < 2:
        return {
            "cagr": np.nan,
            "vol": np.nan,
            "sharpe": np.nan,
            "maxdd": np.nan,
            "calmar": np.nan,
            "skew": np.nan,
            "kurt": np.nan,
        }
    nav = nav.loc[ret.index]
    total_years = len(ret) / periods_per_year
    cagr = (nav.iloc[-1] / nav.iloc[0]) ** (1 / total_years) - 1 if total_years > 0 else np.nan
    vol = ret.std() * np.sqrt(periods_per_year)
    sharpe = ret.mean() / ret.std() * np.sqrt(periods_per_year) if ret.std() > 0 else np.nan
    dd = nav / nav.cummax() - 1.0
    maxdd = dd.min()
    calmar = -cagr / maxdd if maxdd < 0 else np.nan
    return {
        "cagr": cagr,
        "vol": vol,
        "sharpe": sharpe,
        "maxdd": maxdd,
        "calmar": calmar,
        "skew": ret.skew(),
        "kurt": ret.kurtosis(),
    }

def write_log(path: Path, lines):
    def _writer(p):
        with open(p, "w", encoding="utf-8") as f:
            f.write("\n".join(lines) + "\n")
    safe_write(path, _writer)

def save_fig(path: Path):
    def _writer(p):
        plt.savefig(p, dpi=150, bbox_inches="tight")
    safe_write(path, _writer)


In [2]:

# Load panel
panel_path = TABLES_DIR / "panel.pkl"
if panel_path.exists():
    panel = pd.read_pickle(panel_path)
else:
    panel = load_prices_csv(INPUT_CSV)

if "adj_close" not in panel:
    raise ValueError("adj_close not found in panel")

adj_close_raw = panel["adj_close"].copy()
summary = summarize_panel(adj_close_raw)
adj_close, excluded = filter_universe_by_coverage(adj_close_raw, CONFIG["universe_min_coverage"])
panel["adj_close"] = adj_close

ret_d = returns_from_prices(adj_close).fillna(0.0)
px_m = to_month_end(adj_close)
ret_m = returns_from_prices(px_m)

mapping = map_rebalance_to_next_trading_day(px_m.index, adj_close.index)

# Helpers

def zscore(s: pd.Series):
    return (s - s.mean()) / s.std() if s.std() > 0 else s * 0.0

def normalize_longonly(v):
    v = np.clip(v, 0, None)
    s = v.sum()
    if s == 0:
        return v
    return v / s

def normalize_longshort(v):
    denom = np.abs(v).sum()
    if denom == 0:
        return v
    return v / denom

# 01 UMR proxy
mom_12_2 = px_m.shift(CONFIG["mom_skip_m"] + 1) / px_m.shift(CONFIG["mom_lookback_m"]) - 1.0

if "SPY" in ret_d.columns:
    r_mkt = ret_d["SPY"].copy()
else:
    r_mkt = ret_d.mean(axis=1)

# rolling beta (daily)
window = CONFIG["beta_window_d"]
minp = max(20, window // 4)
var = r_mkt.rolling(window, min_periods=minp).var()

beta_cols = {}
for col in ret_d.columns:
    cov = ret_d[col].rolling(window, min_periods=minp).cov(r_mkt)
    beta_cols[col] = cov / var
beta_d = pd.DataFrame(beta_cols)
beta_m = beta_d.resample("ME").last()

w_umr_list = []
for t in px_m.index:
    df = pd.DataFrame({
        "mom": mom_12_2.loc[t],
        "beta": beta_m.loc[t],
    }).dropna()
    if len(df) < 3:
        w = pd.Series(1.0 / adj_close.shape[1], index=adj_close.columns)
    else:
        X = np.column_stack([np.ones(len(df)), df["mom"].values])
        y = df["beta"].values
        try:
            params = np.linalg.lstsq(X, y, rcond=None)[0]
            alpha, gamma = params[0], params[1]
            beta_hat = alpha + gamma * mom_12_2.loc[t]
            w = pd.Series(normalize_longonly(beta_hat.fillna(0.0).values), index=adj_close.columns)
            if w.sum() == 0:
                w = pd.Series(1.0 / adj_close.shape[1], index=adj_close.columns)
        except Exception:
            w = pd.Series(1.0 / adj_close.shape[1], index=adj_close.columns)
    w_umr_list.append(w)

w_umr = pd.DataFrame(w_umr_list, index=px_m.index, columns=adj_close.columns)
res_umr = apply_portfolio(ret_d, w_umr, CONFIG["cost_bps"])

# 02 PPP

def project_to_simplex(v):
    v = np.asarray(v, dtype=float)
    if v.ndim != 1:
        raise ValueError("v must be 1d")
    n = v.size
    if n == 1:
        return np.array([1.0])
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - 1
    ind = np.arange(1, n + 1)
    cond = u - cssv / ind > 0
    if not np.any(cond):
        return np.full_like(v, 1.0 / n)
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / rho
    w = np.maximum(v - theta, 0)
    return w

def pgd_optimize(cov, m_vec, w_prev, eta, gamma, n_iter=200):
    n = len(w_prev)
    w = w_prev.copy()
    try:
        max_eig = np.linalg.eigvalsh(cov).max()
    except Exception:
        max_eig = np.trace(cov) if cov.size else 1.0
    L = max_eig + 2 * gamma + 1e-6
    step = 1.0 / L
    for _ in range(n_iter):
        grad = cov @ w - eta * m_vec + 2 * gamma * (w - w_prev)
        w = w - step * grad
        w = project_to_simplex(w)
    return w

# Sentiment proxy
if "SPY" in px_m.columns and "TLT" in px_m.columns:
    mom6_spy = px_m["SPY"].shift(1) / px_m["SPY"].shift(6) - 1.0
    mom6_tlt = px_m["TLT"].shift(1) / px_m["TLT"].shift(6) - 1.0
    sent = zscore(mom6_spy - mom6_tlt)
else:
    sent = pd.Series(0.0, index=px_m.index)

mom_form = px_m.shift(1) / px_m.shift(CONFIG["mom_form_m"]) - 1.0

dates = px_m.index
tickers = adj_close.columns
n = len(tickers)

w_eqw = []
w_mv = []
w_ppp_mom = []
w_ppp_mom_sent = []

w_prev = np.full(n, 1.0 / n)

for t in dates:
    scores = mom_form.loc[t].dropna()
    if scores.empty:
        m_vec = np.zeros(n)
    else:
        k = max(1, int(np.floor(len(scores) * CONFIG["ppp_winner_pct"])))
        k = min(k, max(1, len(scores)))
        winners = scores.nlargest(k).index
        m_vec = np.zeros(n)
        idx = [tickers.get_loc(x) for x in winners if x in tickers]
        if len(idx) > 0:
            m_vec[idx] = 1.0 / len(idx)

    end_loc = ret_m.index.get_loc(t)
    start_loc = max(0, end_loc - CONFIG["cov_lookback_m"])
    cov = ret_m.iloc[start_loc:end_loc+1].cov().reindex(index=tickers, columns=tickers).fillna(0.0).values

    ones = np.ones(n)
    try:
        inv_cov = np.linalg.pinv(cov)
        w_mv_t = inv_cov @ ones
        w_mv_t = w_mv_t / w_mv_t.sum()
        w_mv_t = project_to_simplex(w_mv_t)
    except Exception:
        w_mv_t = np.full(n, 1.0 / n)

    w_eqw.append(np.full(n, 1.0 / n))
    w_mv.append(w_mv_t)

    w_ppp = pgd_optimize(cov, m_vec, w_prev, CONFIG["ppp_eta"], CONFIG["ppp_gamma"])
    w_ppp_mom.append(w_ppp)

    m_vec_sent = m_vec.copy() if sent.loc[t] > 0 else np.zeros(n)
    w_ppp_sent = pgd_optimize(cov, m_vec_sent, w_prev, CONFIG["ppp_eta"], CONFIG["ppp_gamma"])
    w_ppp_mom_sent.append(w_ppp_sent)

    w_prev = w_ppp_sent

w_eqw = pd.DataFrame(w_eqw, index=dates, columns=tickers)
w_mv = pd.DataFrame(w_mv, index=dates, columns=tickers)
w_ppp_mom = pd.DataFrame(w_ppp_mom, index=dates, columns=tickers)
w_ppp_mom_sent = pd.DataFrame(w_ppp_mom_sent, index=dates, columns=tickers)

res_ppp_eqw = apply_portfolio(ret_d, w_eqw, CONFIG["cost_bps"])
res_ppp_mv = apply_portfolio(ret_d, w_mv, CONFIG["cost_bps"])
res_ppp_mom = apply_portfolio(ret_d, w_ppp_mom, CONFIG["cost_bps"])
res_ppp_mom_sent = apply_portfolio(ret_d, w_ppp_mom_sent, CONFIG["cost_bps"])

# 03 Granger proxy: momentum LS

def build_mom_ls_weights(px_m, lookback, k):
    mom = px_m.shift(1) / px_m.shift(lookback) - 1.0
    w_list = []
    for t in mom.index:
        scores = mom.loc[t].dropna()
        n = len(scores)
        if n < 2:
            w_list.append(pd.Series(0.0, index=px_m.columns))
            continue
        k_eff = min(k, n // 2)
        if k_eff < 1:
            w_list.append(pd.Series(0.0, index=px_m.columns))
            continue
        top = scores.nlargest(k_eff).index
        bot = scores.nsmallest(k_eff).index
        w = pd.Series(0.0, index=px_m.columns)
        w[top] = 1.0 / k_eff
        w[bot] = -1.0 / k_eff
        w_list.append(w)
    return pd.DataFrame(w_list, index=mom.index, columns=px_m.columns)

w_granger = build_mom_ls_weights(px_m, CONFIG["mom_form_m"], CONFIG["k_top_bottom"])
res_granger = apply_portfolio(ret_d, w_granger, CONFIG["cost_bps"])

# 04 Activation
signal = ret_m.ewm(span=CONFIG["activation_span"], min_periods=3).mean()

def activation_deadzone(x, eps):
    ax = np.abs(x)
    return np.where(ax < eps, 0.0, np.sign(x) * (ax - eps))

def activation_tanh(x, k=2.0):
    return np.tanh(k * x)

activations = {
    "activation_linear": lambda x: x,
    "activation_sign": lambda x: np.sign(x),
    "activation_deadzone": lambda x: activation_deadzone(x, 0.2),
    "activation_tanh": lambda x: activation_tanh(x, 2.0),
}

activation_navs = {}
for name, fn in activations.items():
    w_list = []
    for t in signal.index:
        sig_row = pd.Series(fn(signal.loc[t].values), index=signal.columns)
        w = pd.Series(normalize_longshort(sig_row.values), index=signal.columns)
        w_list.append(w)
    w_target = pd.DataFrame(w_list, index=signal.index, columns=signal.columns)
    res = apply_portfolio(ret_d, w_target, CONFIG["cost_bps"])
    activation_navs[name] = res

# 05 JPM trend overlay
trend_scores = []
for h in CONFIG["trend_horizons_m"]:
    score = np.sign(px_m / px_m.shift(h) - 1.0)
    trend_scores.append(score)
trend_score = sum(trend_scores) / len(trend_scores)

vol_d = ret_d.ewm(span=CONFIG["vol_ewma_span"], min_periods=10).std()
vol_m = vol_d.resample("ME").last()

dates = px_m.index
tickers = px_m.columns
w_list = []
nav_approx = [1.0]

for i, t in enumerate(dates):
    score = trend_score.loc[t].fillna(0.0)
    signal = (score > 0).astype(float)
    inv_vol = 1.0 / vol_m.loc[t].replace(0, np.nan)
    w = signal * inv_vol
    if w.sum() > 0:
        w = w / w.sum()
    else:
        w = pd.Series(0.0, index=tickers)

    if i > 0:
        nav_series = pd.Series(nav_approx, index=dates[:len(nav_approx)])
        dd = nav_series / nav_series.cummax() - 1.0
        if dd.min() <= -CONFIG["dd_stop"]:
            w = w * CONFIG["stop_scale"]

    w_list.append(w)
    r_next = ret_m.shift(-1).loc[t].fillna(0.0)
    nav_approx.append(nav_approx[-1] * (1.0 + (w * r_next).sum()))

w_trend = pd.DataFrame(w_list, index=dates, columns=tickers)
res_trend = apply_portfolio(ret_d, w_trend, CONFIG["cost_bps"])

w_eqw_trend = pd.DataFrame(np.full((len(dates), len(tickers)), 1.0 / len(tickers)), index=dates, columns=tickers)
res_trend_eqw = apply_portfolio(ret_d, w_eqw_trend, CONFIG["cost_bps"])

# 06 Comomentum proxy
mom = px_m.shift(1) / px_m.shift(CONFIG["mom_form_m"]) - 1.0
comom_rows = []
window = CONFIG["comom_window_m"]

for i, t in enumerate(mom.index):
    if i < window:
        continue
    scores = mom.loc[t].dropna()
    if len(scores) < 2:
        continue
    k_eff = min(CONFIG["k_top_bottom"], len(scores) // 2)
    if k_eff < 1:
        continue
    losers = scores.nsmallest(k_eff).index
    window_ret = ret_m.iloc[i - window:i][losers]
    mkt = ret_m.iloc[i - window:i].mean(axis=1)
    beta = window_ret.apply(lambda s: s.cov(mkt), axis=0) / mkt.var()
    resid = window_ret - mkt.values.reshape(-1, 1) * beta.values.reshape(1, -1)
    corr = resid.corr()
    if corr.shape[0] < 2:
        continue
    avg_corr = (corr.values[np.triu_indices_from(corr.values, k=1)]).mean()
    comom_rows.append({"date": t, "comom": avg_corr})

comom_df = pd.DataFrame(comom_rows).set_index("date")

# Expanding quantiles for regime
bucket_map = {}
for t in comom_df.index:
    hist = comom_df.loc[:t, "comom"].dropna()
    if len(hist) < 5:
        bucket_map[t] = "mid"
        continue
    q1, q2 = np.quantile(hist, [1/3, 2/3])
    val = comom_df.loc[t, "comom"]
    if val <= q1:
        bucket_map[t] = "low"
    elif val <= q2:
        bucket_map[t] = "mid"
    else:
        bucket_map[t] = "high"

w_mom_ls = build_mom_ls_weights(px_m, CONFIG["mom_form_m"], CONFIG["k_top_bottom"])

w_comom_low = w_mom_ls.copy() * 0.0
w_comom_mid = w_mom_ls.copy() * 0.0
w_comom_high = w_mom_ls.copy() * 0.0

for t, bucket in bucket_map.items():
    if t not in w_mom_ls.index:
        continue
    if bucket == "low":
        w_comom_low.loc[t] = w_mom_ls.loc[t]
    elif bucket == "mid":
        w_comom_mid.loc[t] = w_mom_ls.loc[t]
    else:
        w_comom_high.loc[t] = w_mom_ls.loc[t]

res_comom_low = apply_portfolio(ret_d, w_comom_low, CONFIG["cost_bps"])
res_comom_mid = apply_portfolio(ret_d, w_comom_mid, CONFIG["cost_bps"])
res_comom_high = apply_portfolio(ret_d, w_comom_high, CONFIG["cost_bps"])

# 07 STMOM / MTMOM
w_st = build_mom_ls_weights(px_m, lookback=1, k=CONFIG["k_top_bottom"])
w_mt = build_mom_ls_weights(px_m, lookback=CONFIG["mom_lookback_m"], k=CONFIG["k_top_bottom"])
res_st = apply_portfolio(ret_d, w_st, CONFIG["cost_bps"])
res_mt = apply_portfolio(ret_d, w_mt, CONFIG["cost_bps"])

# Collect NAVs
navs = {
    "umr_beta_proxy": res_umr["nav"],
    "ppp_eqw": res_ppp_eqw["nav"],
    "ppp_mv": res_ppp_mv["nav"],
    "ppp_mom": res_ppp_mom["nav"],
    "ppp_mom_sent": res_ppp_mom_sent["nav"],
    "granger_mom_ls": res_granger["nav"],
    "activation_linear": activation_navs["activation_linear"]["nav"],
    "activation_sign": activation_navs["activation_sign"]["nav"],
    "activation_deadzone": activation_navs["activation_deadzone"]["nav"],
    "activation_tanh": activation_navs["activation_tanh"]["nav"],
    "trend": res_trend["nav"],
    "trend_eqw": res_trend_eqw["nav"],
    "comom_low": res_comom_low["nav"],
    "comom_mid": res_comom_mid["nav"],
    "comom_high": res_comom_high["nav"],
    "stmom": res_st["nav"],
    "mtmom": res_mt["nav"],
}

nav_df = pd.concat(navs, axis=1)

# Align to common period and rebase
nav_df = nav_df.dropna()
if nav_df.empty:
    raise ValueError("No overlapping NAV periods")

nav_df = nav_df / nav_df.iloc[0]

safe_write(PERF_DIR / "nav_compare.csv", lambda p: nav_df.to_csv(p))

plt.figure(figsize=(10, 6))
for col in nav_df.columns:
    plt.plot(nav_df.index, nav_df[col], label=col, linewidth=1)
plt.legend(ncol=2, fontsize=8)
plt.title("NAV comparison (common period, rebased)")
plt.grid(True, alpha=0.3)
save_fig(PLOTS_DIR / "nav_compare.png")
plt.close()

log_lines = []
log_lines.append(f"input_csv: {INPUT_CSV}")
log_lines.append(f"tickers_filtered: {adj_close.shape[1]}")
log_lines.append(f"date_range: {adj_close.index.min().date()} to {adj_close.index.max().date()}")
log_lines.append("\nexcluded_tickers (coverage < min):")
log_lines.append(excluded.to_string())
log_lines.append("\nmonth_end_to_next_trading_day:")
log_lines.append(mapping.to_string(index=False))
log_lines.append("\nstrategies:")
for k in navs.keys():
    log_lines.append(f"- {k}")
log_lines.append("\noutputs:")
log_lines.append(str(PERF_DIR / "nav_compare.csv"))
log_lines.append(str(PLOTS_DIR / "nav_compare.png"))

write_log(LOG_DIR / "08_nav_compare.md", log_lines)
